In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist, everygrams
from itertools import chain
from collections import defaultdict, Counter
    
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HangmanAPI(object):
    def __init__(self):
        self.tries_remaining = 6
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        test_dictionary_location = "words_test.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location) 
        self.test_dictionary = self.build_dictionary(test_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.wins = []
        self.lose_words = []
        
        # building relevant data structures such as substring and individual letter frequencies
        f = open("words_250000_train.txt", "r")
        df = []
        for x in f:
            df.append(x[:-1])

        temp = []
        _2gram = []
        _3gram = []
        _4gram = []    
        _5gram = []

        for word in df:
            temp.append(list(word))         
            _2gram.extend(list(ngrams(word, 2, pad_left=True, pad_right=True)))
            _3gram.extend(list(ngrams(word, 3, pad_left=True, pad_right=True)))
            _4gram.extend(list(ngrams(word, 4, pad_left=True, pad_right=True)))
            _5gram.extend(list(ngrams(word, 5, pad_left=True, pad_right=True)))

        freq_2 = FreqDist(_2gram)
        freq_3 = FreqDist(_3gram)
        freq_4 = FreqDist(_4gram)
        freq_5 = FreqDist(_5gram)

        self.freq_2 = [(elem, freq_2.get(elem)) for elem in freq_2]
        self.freq_3 = [(elem, freq_3.get(elem)) for elem in freq_3]
        self.freq_4 = [(elem, freq_4.get(elem)) for elem in freq_4]
        self.freq_5 = [(elem, freq_5.get(elem)) for elem in freq_5]
        
        self.prev_guess = []
        
        self.vowels = ['a','e','i','o','u']

        self.word_len_dict = {}
        for i in range(3, 30):
            self.word_len_dict[i] = []
            for words in df:
                if(len(words)>i):
                    for j in range(len(words)-i+1):
                        self.word_len_dict[i].append(words[j:j+i]) 
    
                        
        
    def candsort(self, cands, invalids, vowels, vowel_ratio):
        
        # chooses the highest scoring (most common) and valid candidate letter from candidate letters
        for cand in cands:
            if cand[0] == None or cand[0] in invalids:
                continue
            if cand[0] in vowels and vowel_ratio > 0.5:
                continue
            return cand 
        return ('!', 0, 1)
    
    
    def ngram(self, word, index, invalids, freqs, vowel_ratio):
        
        # for each '_' found in the missing word, find valid substring matches and get appropriate weights,
        # giving priority to more complete and longer substrings, tiebreakers between longer substrings are
        # broken by higher weights or frequencies
        
        # candidate tuple structure is: (suggestion, weight, rank) 
        
        freq_2, freq_3, freq_4, freq_5 = freqs
        score1 = ('!', 0, 1)
        score2 = ('!', 0, 1)
        score3 = ('!', 0, 1)
        
        # i. case    
        if index == 0:       
            if word[index+1] == '.':
                return ('!', 0, 1)

            # iXXX Case
            if (len(word) >= 4) and ('.' not in word[index+1:index+4]):
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == None) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2]) and 
                                                                            (elem[0][4] == word[index+3])]
                    return self.candsort(cands, invalids, vowels, vowel_ratio)

            # iXX Case
            if (len(word) >= 3) and ('.' not in word[index+1:index+3]):
                    cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == None) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2])]
                    return self.candsort(cands, invalids, vowels, vowel_ratio)

            # iX case
            cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == None) and 
                                                                    (elem[0][2] == word[index+1])]       
            return self.candsort(cands, invalids, vowels, vowel_ratio)


        # .i case    
        if index == len(word)-1:      
            if word[index-1] == '.':
                return ('!', 0, 1)

            # XXXi case:
            if (len(word) >= 4) and ('.' not in word[index-3:index]):
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][2] == word[index-1]) and 
                                                                            (elem[0][4] == None)]
                    return self.candsort(cands, invalids, vowels, vowel_ratio)         

            # XXi case
            if (len(word) >= 3) and ('.' not in word[index-2:index]):
                    cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][3] == None)]
                    return self.candsort(cands, invalids, vowels, vowel_ratio)    

            # Xi case
            cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                    (elem[0][2] == None)]
            return self.candsort(cands, invalids, vowels, vowel_ratio)


        else:  
            # .i. case
            if word[index-1] == '.' and word[index+1] == '.':
                return ('!', 0, 1)


            # .iX family
            if word[index-1] == '.'and word[index+1] != '.': 


                # .iXXXX case
                if (len(word) - index >= 5) and (index >= 1) and ('.' not in word[index+1:index+5]):
                        cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][2] == word[index+1]) and 
                                                                                (elem[0][3] == word[index+2]) and 
                                                                                (elem[0][4] == word[index+3])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # X.iXX case        
                if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                        cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][3] == word[index+1]) and 
                                                                                (elem[0][4] == word[index+2])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # XX.iX case                
                if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index-1]):
                        cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                                (elem[0][1] == word[index-2]) and 
                                                                                (elem[0][4] == word[index+1])]
                        score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if (score1 != score2) or (score2 != score3) or (score1 != score3):
                    best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]                    


                # .iXX case            
                if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                        cands = [(elem[0][0], elem[1], 3) for elem in freq_3 if (elem[0][1] == word[index+1]) and 
                                                                                (elem[0][2] == word[index+2])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # X.iX case
                if (len(word) - index >= 2) and (index >= 2) and (word[index-2] != '.'):
                        cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][3] == word[index+1])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if score1 != score2:
                    best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]        

                # .iX        
                cands = [(elem[0][0], elem[1], 2) for elem in freq_2 if elem[0][1] == word[index+1]]
                return self.candsort(cands, invalids, vowels, vowel_ratio)


            # Xi. family
            if word[index-1] != '.'and word[index+1] == '.':

                # XXXXi. case
                if (len(word) - index >= 2) and (index >= 4) and ('.' not in word[index-4:index-1]):
                        cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and
                                                                                (elem[0][1] == word[index-2]) and 
                                                                                (elem[0][2] == word[index-1])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # XXi.X case                    
                if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                        cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][1] == word[index-1]) and 
                                                                                (elem[0][4] == word[index+2])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)
                
                # Xi.XX case
                if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index+2:index+4]):
                        cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                                (elem[0][3] == word[index+2]) and 
                                                                                (elem[0][4] == word[index+3])]
                        score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if (score1 != score2) or (score2 != score3) or (score1 != score3):
                    best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]  


                # XXi. case
                if (index >= 2) and (word[index-2] != '.'):
                        cands = [(elem[0][2], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][1] == word[index-1])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)
                
                # Xi.X case
                if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                        cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                                (elem[0][3] == word[index+2])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if score1 != score2:
                    best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]     

                # Xi. case
                cands = [(elem[0][1], elem[1], 2) for elem in freq_2 if elem[0][0] == word[index-1]]
                return self.candsort(cands, invalids, vowels, vowel_ratio)


            # XiX family
            if word[index-1] != '.'and word[index+1] != '.':
                
                # XXiXX case
                if (len(word) - index >= 3) and (index >= 2) and ('.' not in word[index-2:index+3]):
                        cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][1] == word[index-1]) and 
                                                                                (elem[0][3] == word[index+1]) and
                                                                                (elem[0][4] == word[index+2])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # XiXXX case
                if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index-1:index+4]):
                        cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                                (elem[0][2] == word[index+1]) and 
                                                                                (elem[0][3] == word[index+2]) and
                                                                                (elem[0][4] == word[index+3])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # XXXiX case
                if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index+2]):
                        cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                                (elem[0][1] == word[index-2]) and 
                                                                                (elem[0][2] == word[index-1]) and
                                                                                (elem[0][4] == word[index+1])]
                        score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if (score1 != score2) or (score2 != score3) or (score1 != score3):
                    best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]                          


                # XiXX case
                if len(word) - index >= 3 and word[index+2] != '.':
                        cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                                (elem[0][2] == word[index+1]) and 
                                                                                (elem[0][3] == word[index+2])]
                        score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

                # XXiX case
                if index >= 2 and word[index-2] != '.':
                        cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                                (elem[0][1] == word[index-1]) and 
                                                                                (elem[0][3] == word[index+1])]
                        score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

                if score1 != score2:
                    best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                    return best_score[0]            


                # XiX case
                cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                        (elem[0][2] == word[index+1])]
                return self.candsort(cands, invalids, vowels, vowel_ratio)
    
    
    def weighted_count(self, new_dict):
        # count how many times a letter appears in the dictionary
        dictx = collections.Counter()
        for words in new_dict:
            temp = collections.Counter(words)
            for i in temp:
                temp[i] = 1
                dictx += temp
        return dictx
    
    def valid_word(self, word_dict, guess_word):
        # prunes dictionary if training set word does not match length or characters of word-in-progress
        res = []
        for word in word_dict[len(guess_word)]:
            if re.match(guess_word, word):
                res.append(word)
        return res
    
    
    def guess(self, word): # word input example: "_ p p _ e "
        
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        
        clean_word = word[::2].replace("_",".")
        len_word = len(clean_word)
        word_len_dict = self.word_len_dict
        vowels = self.vowels
        
        # update dictionary 
        curr_dict = self.current_dictionary
        new_dict = []
        for dict_word in curr_dict:
            if len(dict_word) != len_word:
                continue           
            if re.match(clean_word, dict_word):
                new_dict.append(dict_word)
                
        self.current_dictionary = new_dict
        temp = self.weighted_count(new_dict)
        letter_weights = temp.most_common()
        
        # filter for failed letters
        failed_letters = []
        for letter in self.guessed_letters:
            if letter not in word:
                failed_letters.append(letter)
        
        # get vowel ratio
        count = 0
        for i in word:
            if i in vowels:
                count += 1
        vowel_ratio = count/len(word)          

        # update the ngrams to remove all failed letters   
        freq_2 = [(elem[0], elem[1]) for elem in self.freq_2 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_3 = [(elem[0], elem[1]) for elem in self.freq_3 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_4 = [(elem[0], elem[1]) for elem in self.freq_4 if set(elem[0]).isdisjoint(set(failed_letters))]
        freq_5 = [(elem[0], elem[1]) for elem in self.freq_5 if set(elem[0]).isdisjoint(set(failed_letters))]
        freqs = [freq_2, freq_3, freq_4, freq_5]

        # ---- Guessing ----
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        
        guess_letter = '!'
        for choice, count in letter_weights:
            if choice not in self.guessed_letters:
                if choice in vowels and vowel_ratio > 0.5:
                    self.guessed_letters.append(choice)
                    continue
                guess_letter = choice
                break
    
        
        # parse through each substring from 3 to half of the length of the target word
        if guess_letter == '!':
            sub_len = round(len_word/2)
            if sub_len >= 3:
                c = collections.Counter()
                for i in range(len_word - sub_len +1):
                    #temp_dict = []
                    #for word in word_len_dict[len_word]:
                    #    if re.match(word, clean_word[i:i+sub_len]):
                    #        temp_dict.append(word)
                    
                    temp_dict = self.valid_word(word_len_dict, clean_word[i:i+sub_len])
                    temp = self.weighted_count(temp_dict)
                    c += temp
                sorted_letter_count = c.most_common()
                
                for letter, _ in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break
                
                        
        # ngram weight guesser
        if guess_letter == '!':
            options = []
            for i in range(len(clean_word)):
                if clean_word[i] == '.':
                    option = self.ngram(clean_word, i, self.guessed_letters, freqs, vowel_ratio)
                    options.append(option)

            best_guesses = sorted(options, key = lambda x: (x[2], x[1]), reverse=True)
            best_guess = best_guesses[0][0]
            if best_guess != '!':
                return best_guess
            
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            char_weights = self.full_dictionary_common_letter_sorted
            for letter, instance_count in char_weights:
                if letter not in self.guessed_letters:
                    if letter in vowels and vowel_ratio > 0.5:
                        self.guessed_letters.append(letter)
                        continue
                    
                    guess_letter = letter
                    break         
                                       
        return guess_letter
    
  
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

                
    def start_game(self, actual_word=None, verbose=True, see_actual=False, train_test = 'train'):
        '''
        Plays a single game of hangman. Specify whether or not to use a word from the training
        dictionary or the test dictionary
        
        Parameters:
            actual_word: specify a word rather than use a random word from the dictionary
            verbose: print out the process throughout the game
            see_actual: preview the word before the game begins
            train_test: choose a random word from the training or test dictionary
        '''
        
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        #self.incorrect_guesses = []
        #self.probabilities = [0] * len(self.letter_set)
        self.tries_remaining = 6
        #self.recalibrate_n_grams()
                 
        if not actual_word:
            if train_test == 'train':
                actual_word = random.choice(self.full_dictionary)
            else:
                actual_word = random.choice(self.test_dictionary)
        
        if verbose and see_actual:
            print('The word is {}'.format(actual_word))
            
        characters = [i for i in actual_word]
        
        masked = ['_' for letter in characters]
    
        word = ' '.join(masked)
    
        while self.tries_remaining > 0:
            
            if verbose:
                print("{} tries remaining ".format(self.tries_remaining), word)                

            
            my_guess = self.guess(word)
            
            self.guessed_letters.append(my_guess)
            
            if verbose:
                print("The letter {} is guessed".format(my_guess))
        
            if my_guess in characters:
                indices = [i for i, x in enumerate(characters) if x == my_guess]
                
                for index in indices:
                    masked[index] = my_guess
                    
                word = ' '.join(masked)
                
                if '_' not in word:
                    if verbose:
                        print(word, 'You win!')
                    self.wins.append(1)
                    break
    
            else:
                self.tries_remaining = self.tries_remaining - 1
                
        if '_' in word:
            if verbose:
                print("You lose!")
            self.wins.append(0)
            self.lose_words.append(actual_word)

In [3]:
from tqdm import tqdm
                        
train = HangmanAPI()
test = HangmanAPI()

for i in tqdm(range(20)):
    train.start_game(verbose=False, train_test='train')
    test.start_game(verbose=False, train_test='test')

print('Train Win % = {0:.0%}'.format(sum(train.wins) / len(train.wins)))
print('Test Win % = {0:.0%}'.format(sum(test.wins) / len(test.wins)))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [11:38<00:00, 34.93s/it]

Train Win % = 95%
Test Win % = 45%
